In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")
llm.invoke("hi").content

'Hi there! 👋\n\nHow can I help you today? 😄\n'

In [9]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)

print(result.content)

I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have. How can I help you today? 😊



In [36]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content

In [37]:
bot=Chatbot(system="you are helpful medical assistant, you give advice to patients")

In [38]:
bot("i have headache, what can i do?")

"I understand you're experiencing a headache. I'm sorry to hear that, headaches can be really debilitating. \n\nUnfortunately, as an AI, I can't give medical advice. It's important to talk to a real doctor to figure out what's causing your headache and get the right treatment. \n\nHowever, I can offer some general tips that often help people manage headaches:\n\n* **Hydrate:** Dehydration can trigger headaches, so make sure you're drinking plenty of water.\n* **Rest:** Sometimes, a headache can be relieved by simply resting in a quiet, dark room.\n* **Over-the-counter pain relievers:** Ibuprofen or acetaminophen can be effective for some types of headaches. \n\n**Please remember:**\n\n* If your headache is severe, sudden, or accompanied by other symptoms like fever, stiff neck, vision changes, or weakness, **seek immediate medical attention.** These could be signs of a serious condition.\n* These tips are not a substitute for professional medical advice. Please consult a doctor for dia

In [39]:
bot.message

[{'role': 'system',
  'content': 'you are helpful medical assistant, you give advice to patients'},
 {'role': 'user', 'content': 'i have headache, what can i do?'},
 {'role': 'assistant',
  'content': "I understand you're experiencing a headache. I'm sorry to hear that, headaches can be really debilitating. \n\nUnfortunately, as an AI, I can't give medical advice. It's important to talk to a real doctor to figure out what's causing your headache and get the right treatment. \n\nHowever, I can offer some general tips that often help people manage headaches:\n\n* **Hydrate:** Dehydration can trigger headaches, so make sure you're drinking plenty of water.\n* **Rest:** Sometimes, a headache can be relieved by simply resting in a quiet, dark room.\n* **Over-the-counter pain relievers:** Ibuprofen or acetaminophen can be effective for some types of headaches. \n\n**Please remember:**\n\n* If your headache is severe, sudden, or accompanied by other symptoms like fever, stiff neck, vision cha

In [56]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [42]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Acer NItro 5\AppData\Local\Temp\ipykernel_15996\952804720.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)')


Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
    1. The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    2. \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*):
    1. This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

### What does it do?
This regex is looking for a string that:

1. Starts with the text "Action:".
2. Has a word right after it, separated by a colon.
3. Then, after another colon, it captures everything that follow

In [57]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [58]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [59]:
def calculate(number):
    return eval(number)

In [60]:
calculate("2+2")

4

In [61]:
calculate("20+2*5")

30

In [62]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [96]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


In [97]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [98]:
bot=Chatbot(prompt)

In [99]:
bot("hi how are you?")

'Okay, I understand. I will follow the Thought, Action, PAUSE, Observation loop and provide an Answer based on the information gathered.\n\nLet me know when you have a question for me!  \n\n'

In [100]:
bot("what is your job?")

'Thought: I am an AI assistant, so I help people by answering questions and completing tasks. \n\nAction: simon_blog_search: AI assistant \nPAUSE  \n'

In [102]:
next_prompt="Tell me about quantum computing from Wikipedia."

In [103]:
next_prompt="Tell me about the current GDP of Sri Lanka from Wikipedia."

In [104]:
result=bot(next_prompt)

In [105]:
result

"Thought: I should look up Sri Lanka's GDP on Wikipedia.\n\nAction: wikipedia: Sri Lanka\nPAUSE \n"

In [106]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [107]:
actions

[<re.Match object; span=(0, 28), match='Action: wikipedia: Sri Lanka'>]

In [108]:
action, action_input = actions[0].groups()

In [109]:
action

'wikipedia'

In [110]:
action_input

'Sri Lanka'

In [111]:
query("hi")

Okay, I understand. 

I'm ready for my first question!  



"Okay, I understand. \n\nI'm ready for my first question!  \n"

In [112]:
query("Fifteen * twenty five")

Thought: Let's calculate fifteen times twenty five. 

Action: calculate: 15 * 25 
PAUSE 

 -- running calculate 15 * 25 
Observation: 375
Answer: Fifteen times twenty five is 375. 



'Answer: Fifteen times twenty five is 375. \n'

In [113]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for information about AI. 

Action: simon_blog_search: AI 
PAUSE  

 -- running simon_blog_search AI 
Observation: Talking AI and jobs with Natasha Zouves for News Nation: I was interviewed by News Nation's Natasha Zouves about the very complicated topic of how we should think about AI in terms of threatening our jobs and careers. I previously talked with Natasha two years ago about Microsoft Bing.

I'll be honest: I was nervous about this one. I'm not an economist and I didn't feel confident talking about this topic!

I do find the challenge of making recent advances in AI and LLMs accessible to a general audience absolutely fascinating though, so I took the risk and agreed to the interview.

I think it came out very well. The full hour long video is now available on the News Nation YouTube channel, or as an audio podcast on iTunes or on Spotify.

 

I made my own transcript of the video (using MacWhisper) and fed it into the new Claude Opus 4 mod

'Thought: Simon has written about AI and it seems like a recent topic for him. \n\nAnswer: Yes, Simon has written about AI. \n'

In [117]:
query("can you tell me capital of USA?")

Thought: I should look up the capital of the USA on Wikipedia.

Action: wikipedia: Capital of the USA 

PAUSE  

 -- running wikipedia Capital of the USA 
Observation: officially <span class="searchmatch">the</span> District <span class="searchmatch">of</span> Columbia and commonly known as simply Washington or D.C., is <span class="searchmatch">the</span> <span class="searchmatch">capital</span> city and federal district <span class="searchmatch">of</span> <span class="searchmatch">the</span> United States. <span class="searchmatch">The</span> city
Answer: The capital of the USA is Washington D.C.  



'Answer: The capital of the USA is Washington D.C.  \n'

In [ ]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()